In [1]:
import cnvutils

In [2]:
import altair as alt
import numpy as np
import os
import statsmodels

from cnvutils.filenames import (
    get_proportions_perm_p_values_path,
)

from cnvutils.constants import SIG_CUTOFF

In [3]:
def prop_perm_p_vals_manhattan_plot(
    chromosome,
    data_dir=os.path.join(os.getcwd(), "..", "data"),
):
    
    p_path = get_proportions_perm_p_values_path(
        data_dir=data_dir,
        chromosome=chromosome,
    )
    
    p_vals = pd.read_csv(p_path, sep="\t")
    
    # Filter out just the p values we want
    p_vals = p_vals[p_vals["group"] == "sig_props"][["name", "p"]]
    p_vals = p_vals[p_vals["name"].str.contains("gistic")]
    
    # Benjamini-Hochberg FDR correction
    reject, adj_p, alphac_sidak, alphac_bonf = statsmodels.stats.multitest.multipletests(
        pvals=p_vals["p"],
        alpha=SIG_CUTOFF,
        method="fdr_bh",
    )
    
    p_vals = p_vals.assign(adj_p=adj_p)
    
#     return p_vals

    # Make Manhattan plot
    p_vals = p_vals.\
    assign(
        line=-np.log10(SIG_CUTOFF),
        neg_log_adj_p=-np.log10(p_vals["adj_p"]),
    ).\
    drop(columns="p")

    base = alt.Chart(p_vals)

    dots = base.mark_point().encode(
        x="name",
        y="neg_log_adj_p",
    )

    line = base.mark_rule(color="crimson").encode(
        y="line"
    )

    return dots + line

prop_perm_p_vals_manhattan_plot(chromosome=8)

alt.LayerChart(...)